In [1]:
from pulp import *
import numpy as np

In [2]:
materials = [115, 115, 220, 180, 112, 112]
required_lengths = [40, 36.4, 34]
frequencies = [11, 6, 2]
cut_width = 1.5

In [3]:
"; ".join([f"{j} x {i}m" for i, j in zip(required_lengths, frequencies)])

'11 x 40m; 6 x 36.4m; 2 x 34m'

In [22]:
def optim_cut(woods, num_pieces, required_lengths, frequencies, cut_width):
    pieces = [("piece{}.{}".format(woods[i], j), woods[i]) for i in range(len(woods)) for j in range(num_pieces[i])]
    lens = [f'len{l}' for l in required_lengths]
    
    pieces_var = [dict(
        val = p[1], 
        len_var = pulp.LpVariable.dicts(p[0], lens, lowBound = 0, cat = "Integer"),
        leftover_var = LpVariable(f"{p[0]}_leftover", lowBound = 0, cat = "Continuous")
    )  for p in pieces]
    n = LpVariable("n", lowBound = 0, cat = "Integer")
    leftover_max = LpVariable("leftover_max", lowBound = 0, upBound = max(materials), cat = "Continuous")
    
    prob = LpProblem("wood_cut", LpMaximize)
    prob += lpSum([10e5 * n, leftover_max]) 
    
    for p in pieces_var:
        prob += lpSum([i * (j + cut_width) for i, j in zip(p['len_var'].values(), required_lengths)] +
                    p['leftover_var']
                   ) == p['val']
        prob += lpSum([p['leftover_var'], -leftover_max]) >= 0
    
    for l, f in zip(lens, frequencies):
        prob += lpSum([f * n] + [-p['len_var'][l] for p in pieces_var])  <= 0
    prob.solve(CPLEX_PY())
    return (pieces_var, int(n.varValue))

In [23]:
materials = [200] * 10 + [150] * 5
(woods, num_pieces) = np.unique(materials, return_counts = True)
(pieces_res, n_res) = optim_cut(woods, num_pieces, required_lengths, frequencies, cut_width)

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
Aggregator did 15 substitutions.
Reduced MIP has 18 rows, 47 columns, and 108 nonzeros.
Reduced MIP has 0 binaries, 46 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.10 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 18 rows, 47 columns, and 108 nonzeros.
Reduced MIP has 0 binaries, 46 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.09 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.08 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000  5000150.0000       

In [60]:
res, freq = np.unique([("{}m-length piece: ".format(p['val']) +  
            '; '.join(["{} x {}m".format(int(j.varValue), i[3:])for i,j in p['len_var'].items() if j.varValue > 0])+ 
            "({}m left)".format(p['leftover_var'].varValue))
           for p in pieces_res],
          return_counts = True)

In [65]:
import re

In [72]:
[re.sub(': ', f": {j} ", i)  for i, j in zip (res, list(map(lambda k: f'{k} times' if k > 1 else '', freq)))]

['150m-length piece:  2 x 40m; 1 x 34m(31.5m left)',
 '150m-length piece:  2 x 40m; 1 x 36.4m(29.099999999999994m left)',
 '150m-length piece:  3 x 36.4m(36.30000000000001m left)',
 '150m-length piece: 2 times 3 x 40m(25.5m left)',
 '200m-length piece:  3 x 40m; 1 x 36.4m(37.599999999999994m left)',
 '200m-length piece: 2 times 4 x 36.4m(48.400000000000006m left)',
 '200m-length piece: 5 times 4 x 40m(34.0m left)',
 '200m-length piece:  5 x 34m(22.5m left)',
 '200m-length piece:  5 x 36.4m(10.5m left)']

In [24]:
for p in pieces_res:
    print(f"{p['val']}m-length piece: ")
    print('; '.join(["{} x {}m".format(int(j.varValue), i[3:])for i,j in p['len_var'].items() if j.varValue > 0]))
    print("({}m left)".format(p['leftover_var'].varValue))

150m-length piece: 
2 x 40m; 1 x 34m
(31.5m left)
150m-length piece: 
3 x 36.4m
(36.30000000000001m left)
150m-length piece: 
2 x 40m; 1 x 36.4m
(29.099999999999994m left)
150m-length piece: 
3 x 40m
(25.5m left)
150m-length piece: 
3 x 40m
(25.5m left)
200m-length piece: 
3 x 40m; 1 x 36.4m
(37.599999999999994m left)
200m-length piece: 
5 x 34m
(22.5m left)
200m-length piece: 
4 x 40m
(34.0m left)
200m-length piece: 
4 x 40m
(34.0m left)
200m-length piece: 
4 x 40m
(34.0m left)
200m-length piece: 
4 x 36.4m
(48.400000000000006m left)
200m-length piece: 
4 x 40m
(34.0m left)
200m-length piece: 
5 x 36.4m
(10.5m left)
200m-length piece: 
4 x 40m
(34.0m left)
200m-length piece: 
4 x 36.4m
(48.400000000000006m left)


In [7]:
lens = [f'len{l}' for l in required_lengths]

In [8]:
(woods, num_pieces) = np.unique(materials, return_counts = True)
pieces = [("piece{}.{}".format(woods[i], j), woods[i]) for i in range(len(woods)) for j in range(num_pieces[i])]

In [9]:
pieces_var = [dict(
    val = p[1], 
    len_var = pulp.LpVariable.dicts(p[0], lens, lowBound = 0, cat = "Integer"),
    leftover_var = LpVariable(f"{p[0]}_leftover", lowBound = 0, cat = "Continuous")
)  for p in pieces]

In [10]:
n = LpVariable("n", lowBound = 0, cat = "Integer")
leftover_max = LpVariable("leftover_max", lowBound = 0, upBound = max(materials), cat = "Continuous")

In [11]:
prob = LpProblem("wood_cut", LpMaximize)

In [12]:
prob += lpSum([10e5 * n, leftover_max]) 

In [13]:
for p in pieces_var:
    prob += lpSum([i * (j + cut_width) for i, j in zip(p['len_var'].values(), required_lengths)] +
                p['leftover_var']
               ) == p['val']
    prob += lpSum([-p['leftover_var'], leftover_max]) >= 0

In [14]:
for l, f in zip(lens, frequencies):
    prob += lpSum([f * n] + [-p['len_var'][l] for p in pieces_var])  <= 0

In [15]:
prob.solve()

1

In [16]:
n.varValue

1.0

In [17]:
CPLEX_PY()

In [18]:
listSolvers(onlyAvailable=True)

/Users/minh/opt/anaconda3/lib/python3.7/site-packages/pulp/apis/gurobi_api.py:357: UserWarning: GUROBI error: Set parameter LogFile to value "gurobi.log"

Failed to set up a license

Error 10009: No Gurobi license found (user minh, host minhs-mbp.home, hostid c22568c4, cores 4)


.
  warnings.warn("GUROBI error: {}.".format(out))


['CPLEX_PY', 'PULP_CBC_CMD']

In [ ]:
w = [115, 115, 220, 180, 112, 112]
p = [40, 34, 36.4]
f = [11, 2, 6]
c = 1.5

In [ ]:
n_var_each = len(p) + 1
n_mat = len(w)

In [ ]:
(wood, num) = np.unique(w, return_counts = True)
name_var = sum([["{}_piece{}_{}".format(k, wood[i], j) for j in range(num[i]) \
                 for k in ["leftover"] + ["len{}".format(i) for i in p]] for i in range(len(wood))], []) + ['n', 'leftover_max']

In [ ]:
x_var = [LpVariable(i, lowBound = 0, cat = j) for i, j in zip(name_var, type_var)]
n_var = len(x_var)

In [ ]:
coef_var = np.zeros(n_var)

In [ ]:
prob = LpProblem("test", LpMaximize)

In [ ]:
coef_obj = coef_var.copy()

coef_obj[-2] = 1

prob += lpSum([x_var[i] * coef_obj[i] for i in range(n_var)])

In [ ]:
coef_ini = [[1] + [i + c for i in p]]
for piece in range(n_mat):
    
    indices = [i + piece * n_var_each for i in range(n_var_each)]
    coef_full = coef_var.copy()
    coef_full[indices] = coef_ini
    prob += lpSum([x_var[i] * coef_full[i] for i in range(n_var)]) == w[piece]
    

In [ ]:
for t in range(n_var_each - 1):
    constr = coef_var.copy()
    constr[[i for i in range(n_var - 1) if i % n_var_each == (t + 1)]] = -1
    constr[-2] = f[t]
    prob += lpSum([x_var[i] * constr[i] for i in range(n_var)]) <= 0

In [ ]:
prob

In [ ]:
prob.solve()

In [ ]:
prob.objective

In [ ]:
prob.objective.value()

In [ ]:
x_soln = np.array([x_var[i].varValue for i in range(n_var)])

In [ ]:
print(["{}: {}".format(j, i) for i, j in zip(x_soln, name_var)])

In [75]:
n = 6
[i * n for i in [11, 6, 2, 1]]

[66, 36, 12, 6]